# 27 Juni 2021 Supertrend strategy probeersel

Gisteren en vandaag heerlijk gefietst met Vita en Nastja langs de Kaag, Brasemermeer, verschillende rivieren etcetera. Prachtige fietsdag met volop zon en warm en... Prachtige huizen, boten, campers, motors en meer...

Dit geeft me weer inspiratie om door te gaan, op zoek naar een nieuwe strategie die werkt!

Mijn nieuwe uitgangspunten voor deze strategie zijn als volgt:
* Laat je niet gek maken met onrealistische winsten die onrealistisch te voorspellen zijn. Gebruik Top5 coins en een strategie die daarop betrouwbaar is.
* De strategie moet echt een winstpercentage van 50% of hoger hebben (minstens even vaak winnen als verliezen)!
* Timeframe maakt me niet uit, als het W/L percentage maar minstens 50% is.
* Swingtrade of TP punten is arbitrair. Misschien wel beiden waarvan de eerste die gehaald wordt de eerste TP is.
* Stoploss niet afhankelijk maken van percentages want dat schiet niet op...

## Coins die gebruikt gaan worden:

Top5 coins op Coinmarketcap:
* Bitcoin
* Ethereum
* Binance coin
* Cardano
* Dogecoin (WAT??)
* Ripple (reserve)

### Meer info

http://localhost:8888/notebooks/freqtrade-supertrend.ipynb

# Rules of Engagement

## Indicatoren
**Timeframe:** Beginnen met 30 min
**ATR Supertrend Ultra:** ATR Period: 7; Multiplier: 2.5
6
**QQE:** RSI Length: 7; Smooting: 5; Qup/down: 4.236

## Trading rules

Entry

* ATR: veranderd in bullish signaal (alleen op dit signaal instappen en niet ergens in die bullish trend)
* QQE: RSI is boven RSI Smooth factor EN RSI is alleen tussen 50 en 65 (niet hoger en niet lager instappen)

(Trailing) Stoploss

* ATR bullish niveau (de atr lijn onder de candle)

Exit

* R:R is 1:1,3 op Stoploss of
* Close onder stoploss ATR lijn



## Pseudocode

```
if atr_supert_direction == 1 (of True?)
    if qqe_rsi > qqe_rsi_sma && qqe_rsi > 50
    
SL = supert

TP = supert ||  ((entry - supert) * 1,3)
```

# Indicatoren

## Supertrend

In [131]:
# Importeer de benodigde bibliotheken
import pandas as pd
import pandas_ta as pta
from pandas import DataFrame
import numpy as np

In [132]:
# Load data
df = pd.read_json("./data/ETH_USDT-30m.json")
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.tail(5)

,date,open,high,low,close,volume
67375,2021-06-26 05:30:00,1837.53,1837.66,1826.66,1832.02,4871.99841
67376,2021-06-26 06:00:00,1832.03,1843.32,1828.77,1836.09,6322.17702
67377,2021-06-26 06:30:00,1836.21,1838.20,1802.00,1806.10,21995.33935
67378,2021-06-26 07:00:00,1806.14,1810.24,1760.70,1767.50,45382.78487
67379,2021-06-26 07:30:00,1767.31,1771.71,1745.00,1752.00,37353.11390


In [133]:
help(pta.supertrend)

Help on function supertrend in module pandas_ta.overlap.supertrend:

supertrend(high, low, close, length=None, multiplier=None, offset=None, **kwargs)
    Supertrend (supertrend)
    
    Supertrend is an overlap indicator. It is used to help identify trend
    direction, setting stop loss, identify support and resistance, and/or
    generate buy & sell signals.
    
    Sources:
        http://www.freebsensetips.com/blog/detail/7/What-is-supertrend-indicator-its-calculation
    
    Calculation:
        Default Inputs:
            length=7, multiplier=3.0
        Default Direction:
            Set to +1 or bullish trend at start
    
        MID = multiplier * ATR
        LOWERBAND = HL2 - MID
        UPPERBAND = HL2 + MID
    
        if UPPERBAND[i] < FINAL_UPPERBAND[i-1] and close[i-1] > FINAL_UPPERBAND[i-1]:
            FINAL_UPPERBAND[i] = UPPERBAND[i]
        else:
            FINAL_UPPERBAND[i] = FINAL_UPPERBAND[i-1])
    
        if LOWERBAND[i] > FINAL_LOWERBAND[i-1] and clos

In [134]:
pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=7, multiplier=2.5).tail(5)

,SUPERT_7_2.5,SUPERTd_7_2.5,SUPERTl_7_2.5,SUPERTs_7_2.5
67375,1879.758093,-1,NaN,1879.758093
67376,1879.758093,-1,NaN,1879.758093
67377,1875.633631,-1,NaN,1875.633631
67378,1850.763113,-1,NaN,1850.763113
67379,1823.859811,-1,NaN,1823.859811


In [135]:
# Supertrend opnemen in dataframe
df['st'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=7, multiplier=2.5)['SUPERTd_7_2.5']
# df.drop(['supertrend'], axis = 1)
df.tail(5)

,date,open,high,low,close,volume,st
67375,2021-06-26 05:30:00,1837.53,1837.66,1826.66,1832.02,4871.99841,-1
67376,2021-06-26 06:00:00,1832.03,1843.32,1828.77,1836.09,6322.17702,-1
67377,2021-06-26 06:30:00,1836.21,1838.20,1802.00,1806.10,21995.33935,-1
67378,2021-06-26 07:00:00,1806.14,1810.24,1760.70,1767.50,45382.78487,-1
67379,2021-06-26 07:30:00,1767.31,1771.71,1745.00,1752.00,37353.11390,-1


### Supertrend signaal

Als supertrend bullish wordt, dan een instapsignaal genereren. Niet daarvoor en niet meer daarna.

In [136]:
df['st_signal'] = (df['st'].shift(+1) != df['st'])&(df['st'] == 1)
df[['date','st_signal']][df['st_signal']==True].tail(15)

,date,st_signal
66342,2021-06-04 17:00:00,True
66406,2021-06-06 01:00:00,True
66452,2021-06-07 00:00:00,True
66539,2021-06-08 19:30:00,True
66561,2021-06-09 06:30:00,True
66581,2021-06-09 16:30:00,True
66615,2021-06-10 09:30:00,True
66713,2021-06-12 10:30:00,True
66772,2021-06-13 16:00:00,True
66936,2021-06-17 02:00:00,True


## QQE

In [137]:
help(pta.qqe)

Help on function qqe in module pandas_ta.momentum.qqe:

qqe(close, length=None, smooth=None, factor=None, mamode=None, drift=None, offset=None, **kwargs)
    Quantitative Qualitative Estimation (QQE)
    
    The Quantitative Qualitative Estimation (QQE) is similar to SuperTrend but uses a Smoothed RSI with an upper and lower bands. The band width is a combination of a one period True Range of the Smoothed RSI which is double smoothed using Wilder's smoothing length (2 * rsiLength - 1) and multiplied by the default factor of 4.236. A Long trend is determined when the Smoothed RSI crosses the previous upperband and a Short trend when the Smoothed RSI crosses the previous lowerband.
    
    Based on QQE.mq5 by EarnForex Copyright © 2010, based on version by Tim Hyder (2008), based on version by Roman Ignatov (2006)
    
    Sources:
        https://www.tradingview.com/script/IYfA9R2k-QQE-MT4/
        https://www.tradingpedia.com/forex-trading-indicators/quantitative-qualitative-estimati

In [138]:
pta.qqe(close=df['close'], length=7, smooth=None, factor=None, mamode=None, drift=None, offset=None).tail(5)

,QQE_7_5_4.236,QQE_7_5_4.236_RSIMA,QQEl_7_5_4.236,QQEs_7_5_4.236
67375,44.127303,53.429229,44.127303,NaN
67376,45.725070,54.773114,45.725070,NaN
67377,45.725070,48.332309,45.725070,NaN
67378,49.649268,39.723442,NaN,49.649268
67379,43.711004,32.888331,NaN,43.711004


### QQE Signaal

QQE bepalen en QQE signal bepalen.

LET OP: qqe_rsi(ma) is de signal lijn en als deze boven of onder de qqe lijn komt, dan geeft deze bullish of bearish signalen.

* sig1 = qqe_sma > qqe 
* sig2 = qqe_sma > 50

Als beiden True zijn, dan is qqe_signal ook True.

In [139]:
# QQE
df['qqe']=pta.qqe(close=df['close'], length=7, smooth=None, factor=None, mamode=None, drift=None, offset=None)['QQE_7_5_4.236']
df['qqe_rsi']=pta.qqe(close=df['close'], length=7, smooth=None, factor=None, mamode=None, drift=None, offset=None)['QQE_7_5_4.236_RSIMA']
# QQE signal precondition 1: QQE moet boven qqe sma zijn
df['qqe_sig1'] = (df['qqe_rsi'] > df['qqe'])
# QQE signal precondition 2: QQE moet boven 50 zijn
df['qqe_sig2'] = (df['qqe_rsi'] > 50)
# Als beide condities True zijn, dan QQE signal is ook True
df['qqe_signal'] = (df['qqe_sig1'] == True)&(df['qqe_sig2'] == True)
df[['date','close','st','st_signal','qqe_signal','qqe','qqe_rsi']].tail(15)

,date,close,st,st_signal,qqe_signal,qqe,qqe_rsi
67365,2021-06-26 00:30:00,1836.87,-1,False,False,49.960183,44.044020
67366,2021-06-26 01:00:00,1810.78,-1,False,False,49.960183,42.514307
67367,2021-06-26 01:30:00,1824.18,-1,False,False,49.960183,44.121142
67368,2021-06-26 02:00:00,1811.48,-1,False,False,49.960183,43.207382
67369,2021-06-26 02:30:00,1820.71,-1,False,False,49.960183,44.481639
67370,2021-06-26 03:00:00,1823.05,-1,False,False,49.960183,45.820567
67371,2021-06-26 03:30:00,1819.34,-1,False,False,49.960183,45.924738
67372,2021-06-26 04:00:00,1818.80,-1,False,False,49.960183,45.867874
67373,2021-06-26 04:30:00,1834.86,-1,False,False,49.960183,49.840393
67374,2021-06-26 05:00:00,1837.53,-1,False,True,43.521881,53.069112


In [140]:
df[['date','close','st','st_signal','qqe_signal','qqe','qqe_rsi']].tail(15)

,date,close,st,st_signal,qqe_signal,qqe,qqe_rsi
67365,2021-06-26 00:30:00,1836.87,-1,False,False,49.960183,44.044020
67366,2021-06-26 01:00:00,1810.78,-1,False,False,49.960183,42.514307
67367,2021-06-26 01:30:00,1824.18,-1,False,False,49.960183,44.121142
67368,2021-06-26 02:00:00,1811.48,-1,False,False,49.960183,43.207382
67369,2021-06-26 02:30:00,1820.71,-1,False,False,49.960183,44.481639
67370,2021-06-26 03:00:00,1823.05,-1,False,False,49.960183,45.820567
67371,2021-06-26 03:30:00,1819.34,-1,False,False,49.960183,45.924738
67372,2021-06-26 04:00:00,1818.80,-1,False,False,49.960183,45.867874
67373,2021-06-26 04:30:00,1834.86,-1,False,False,49.960183,49.840393
67374,2021-06-26 05:00:00,1837.53,-1,False,True,43.521881,53.069112


### Definitieve signaal voor entry

Dit zijn de definitieve signalen voor entry!

Als (qqe_signal == True) en (st_signal == True)

Dan entry signal

In [141]:
df['entry'] = (df['st_signal'] == True) & (df['qqe_signal'] == True)
df[['date','close','st','st_signal','qqe','qqe_rsi','qqe_sig1','qqe_sig2','qqe_signal','entry']][df['entry']==True].tail(15)

,date,close,st,st_signal,qqe,qqe_rsi,qqe_sig1,qqe_sig2,qqe_signal,entry
66269,2021-06-03 04:30:00,2767.17,1,True,48.910256,61.138454,True,True,True,True
66342,2021-06-04 17:00:00,2700.00,1,True,47.236513,59.119839,True,True,True,True
66406,2021-06-06 01:00:00,2665.32,1,True,46.819728,57.928453,True,True,True,True
66452,2021-06-07 00:00:00,2747.55,1,True,47.239980,60.336129,True,True,True,True
66539,2021-06-08 19:30:00,2501.15,1,True,44.252401,59.098608,True,True,True,True
66561,2021-06-09 06:30:00,2532.82,1,True,46.286511,60.386986,True,True,True,True
66581,2021-06-09 16:30:00,2597.01,1,True,49.265115,62.202888,True,True,True,True
66713,2021-06-12 10:30:00,2393.54,1,True,57.931536,68.670117,True,True,True,True
66772,2021-06-13 16:00:00,2392.00,1,True,40.539994,53.177936,True,True,True,True
66936,2021-06-17 02:00:00,2425.55,1,True,42.577000,54.655922,True,True,True,True


In [142]:
df[['date','close','entry']][df['entry']==True].tail(15)

,date,close,entry
66269,2021-06-03 04:30:00,2767.17,True
66342,2021-06-04 17:00:00,2700.00,True
66406,2021-06-06 01:00:00,2665.32,True
66452,2021-06-07 00:00:00,2747.55,True
66539,2021-06-08 19:30:00,2501.15,True
66561,2021-06-09 06:30:00,2532.82,True
66581,2021-06-09 16:30:00,2597.01,True
66713,2021-06-12 10:30:00,2393.54,True
66772,2021-06-13 16:00:00,2392.00,True
66936,2021-06-17 02:00:00,2425.55,True


## Takeprofit punt bepalen

Take profit is een factor dat aangepast kan worden. Initieel is dit 1,3x de afstand tot initial stoploss punt voor R:R  = 1:1,3

**Deze berekening werkt nu niet goed maar als ik in de strategie eerst kijk of uitstappen wanneer close is onder supertrend...***

In [153]:
# Initial stoploss is waarde van supertrend op het moment van omslag naar bullish

# Rekenfactor voor takeprofit
# Idee dat bij mij opgekomen is. De TP factor kan ik aanpassen als er bepaalde condities zijn dit dit noodzakelijk maken.
# Bijvoorbeeld als QQE al erg hoog is, dan kan sl_fact niet 1.3 zijn maar bijv. 1.0 zodat eerder een zekere winst te behalen valt.
df['sl_fact'] = 1.3

# (Trailing) stop loss
df['sl'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=7, multiplier=2.5)['SUPERT_7_2.5']

# Als Entry == True, bereken dan takeprofit obv. stoploss en stoploss factor.
# Anders, pak vorige rijwaarde takeprofit
# for i, row in df.iterrows():
#     if row['entry'] == True:
#         df.loc[i,'tp'] = ((row['close']-row['sl']) * row['sl_fact']) + row['close']
#     else:
#         df.loc[i,'tp'] = df.loc[i,'tp']
        
# df[['date','close','entry','sl','sl_fact','tp']][df['entry']==True].tail(5)
df[['date','close','entry','sl','sl_fact','tp']][df['entry']==True].tail(15)

,date,close,entry,sl,sl_fact,tp
66269,2021-06-03 04:30:00,2767.17,True,2684.780484,1.3,2874.276371
66342,2021-06-04 17:00:00,2700.00,True,2581.889293,1.3,2853.543919
66406,2021-06-06 01:00:00,2665.32,True,2566.477336,1.3,2793.815464
66452,2021-06-07 00:00:00,2747.55,True,2651.550041,1.3,2872.349947
66539,2021-06-08 19:30:00,2501.15,True,2378.256112,1.3,2660.912055
66561,2021-06-09 06:30:00,2532.82,True,2425.253562,1.3,2672.656369
66581,2021-06-09 16:30:00,2597.01,True,2472.437326,1.3,2758.954477
66713,2021-06-12 10:30:00,2393.54,True,2293.948421,1.3,2523.009053
66772,2021-06-13 16:00:00,2392.00,True,2310.728735,1.3,2497.652644
66936,2021-06-17 02:00:00,2425.55,True,2363.209841,1.3,2506.592207


In [152]:
# df[['date','close','entry','sl','sl_fact','tp']][df['entry']==True].tail(100).to_csv(r'eth_30m_trades.csv')
df[['date','close','entry','sl','sl_fact','tp','qqe_signal','qqe_rsi','qqe']][df['entry']==True].tail(100)

,date,close,entry,sl,sl_fact,tp,qqe_signal,qqe_rsi,qqe
60628,2021-02-05 07:00:00,1666.33,True,1603.531893,1.3,1747.967539,True,63.015713,52.312504
60700,2021-02-06 19:00:00,1714.09,True,1664.844443,1.3,1778.109224,True,61.097921,51.594512
60753,2021-02-07 21:30:00,1596.76,True,1531.446571,1.3,1681.667457,True,55.395349,40.571530
60857,2021-02-10 01:30:00,1803.74,True,1746.841154,1.3,1877.708499,True,69.938276,60.634116
60919,2021-02-11 09:30:00,1765.94,True,1720.595142,1.3,1824.888316,True,70.745583,59.816806
...,...,...,...,...,...,...,...,...,...
67028,2021-06-19 00:00:00,2249.65,True,2171.372843,1.3,2351.410304,True,59.901695,45.919549
67052,2021-06-19 12:00:00,2258.64,True,2195.902708,1.3,2340.198480,True,57.888729,46.529400
67112,2021-06-20 18:00:00,2144.95,True,2084.906647,1.3,2223.006358,True,59.284918,43.279665
67204,2021-06-22 16:00:00,1910.20,True,1742.646931,1.3,2128.018990,True,52.662217,37.434125


Hieronder de mogelijke permanente oplossing voor het behouden van een takeprofit level. Initieel wordt de waarde TP gevuld op het moment dat entry==True. Echter daarna is deze niet meer true en komt er dus geen waarde meer in de cel te staan, terwijl we deze wel van de initiele situatie willen behouden. Cel heeft daarom NaN.

Gelukkig is er de forward fill method die alle NaN's vult met de laatst bekende waarden. Daardoor hoef je dus niet moeilijk te doen met loc[i-1] die errors geeft zoals invalid vaue -1 etcetera!

Waarom deze in eerdere scripts wel werkte maar hier niet is mij een raadsel...

In [161]:
# Als Entry == True, bereken dan takeprofit obv. stoploss en stoploss factor.
# Anders, pak vorige rijwaarde takeprofit
for i, row in df.iterrows():
    if row['entry'] == True:
        df.loc[i,'tp'] = ((row['close']-row['sl']) * row['sl_fact']) + row['close']
    else:
        df.loc[i,'tp'] = df.fillna(method='ffill', inplace=True)
        
# df[['date','close','entry','sl','sl_fact','tp']][df['entry']==True].tail(15)
df[['date','close','entry','sl','sl_fact','tp']].tail(15)

,date,close,entry,sl,sl_fact,tp
67365,2021-06-26 00:30:00,1836.87,False,1882.162546,1.3,2042.92541
67366,2021-06-26 01:00:00,1810.78,False,1882.162546,1.3,2042.92541
67367,2021-06-26 01:30:00,1824.18,False,1882.162546,1.3,2042.92541
67368,2021-06-26 02:00:00,1811.48,False,1882.162546,1.3,2042.92541
67369,2021-06-26 02:30:00,1820.71,False,1881.865352,1.3,2042.92541
67370,2021-06-26 03:00:00,1823.05,False,1881.865352,1.3,2042.92541
67371,2021-06-26 03:30:00,1819.34,False,1881.865352,1.3,2042.92541
67372,2021-06-26 04:00:00,1818.80,False,1879.758093,1.3,2042.92541
67373,2021-06-26 04:30:00,1834.86,False,1879.758093,1.3,2042.92541
67374,2021-06-26 05:00:00,1837.53,False,1879.758093,1.3,2042.92541


In [160]:
df[['date','close','entry','sl','sl_fact','tp']].tail(55)

,date,close,entry,sl,sl_fact,tp
67325,2021-06-25 04:30:00,1977.46,False,1964.864693,1.3,2042.92541
67326,2021-06-25 05:00:00,1969.99,False,1964.864693,1.3,2042.92541
67327,2021-06-25 05:30:00,1964.28,False,2020.462813,1.3,2042.92541
67328,2021-06-25 06:00:00,1964.51,False,2017.608125,1.3,2042.92541
67329,2021-06-25 06:30:00,1967.01,False,2015.429822,1.3,2042.92541
67330,2021-06-25 07:00:00,1934.29,False,2006.653418,1.3,2042.92541
67331,2021-06-25 07:30:00,1944.06,False,1992.210073,1.3,2042.92541
67332,2021-06-25 08:00:00,1930.39,False,1987.475777,1.3,2042.92541
67333,2021-06-25 08:30:00,1934.08,False,1987.475777,1.3,2042.92541
67334,2021-06-25 09:00:00,1936.37,False,1987.475777,1.3,2042.92541
